In [8]:
import pandas as pd


In [9]:
#load the data set from physionet
df = pd.read_csv("orchid.csv")

C:\Users\adams\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [10]:
# Convert to timestamp columns
time_columns = ['time_referred', 'time_approached', 'time_authorized', 'time_procured']
for col in time_columns:
    df[col] = pd.to_datetime(df[col])

In [11]:
from pandas import Timedelta
event_log_list = []
# Iterate over each row to extract events and their timestamps
for i, row in df.iterrows():
    patient_id = row['PatientID']
    event_rows = []
    opo = row["OPO"]
    gender = row["Gender"]
    race = row["Race"]
    age = row["Age"]
    # Add each activity only if its corresponding entry is not False
    # If there are no timestamps for an event we use the timestamp of the next event and subtract one second
    if row['time_referred']:
        event_rows.append({'PatientID': patient_id, 'Activity': 'Referral', 'Timestamp': row['time_referred'], "OPO":opo, "Gender":gender,"Race":race,"Age":age})
        
    if row['time_referred']:  # Assuming evaluation happens after referral
        event_rows.append({'PatientID': patient_id, 'Activity': 'Evaluation', 'Timestamp': row['time_referred'] + Timedelta(minutes=1), "OPO":opo, "Gender":gender,"Race":race,"Age":age})
        
    if row['approached']:
        event_rows.append({'PatientID': patient_id, 'Activity': 'Approach', 'Timestamp': row['time_approached'], "OPO":opo, "Gender":gender,"Race":race,"Age":age})
        
    if row['authorized']:
        event_rows.append({'PatientID': patient_id, 'Activity': 'Authorization', 'Timestamp': row['time_authorized'], "OPO":opo, "Gender":gender,"Race":race,"Age":age})
        
    if row['procured']:
        event_rows.append({'PatientID': patient_id, 'Activity': 'Procurement', 'Timestamp': row['time_procured'], "OPO":opo, "Gender":gender,"Race":race,"Age":age})
        
    if row['transplanted']:
        event_rows.append({'PatientID': patient_id, 'Activity': 'Transplant', 'Timestamp': row['time_procured'] + Timedelta(minutes=1), "OPO":opo, "Gender":gender,"Race":race,"Age":age})
    
    if event_rows:
        temp_df = pd.DataFrame(event_rows)
        event_log_list.append(temp_df)

# Concatenate all DataFrame chunks to make the final event log
event_log = pd.concat(event_log_list, ignore_index=True)

# Sort event log by PatientID and Timestamp
event_log.sort_values(by=['PatientID', 'Timestamp'], inplace=True)

# Save the event log to a CSV file
event_log.to_csv("event_log.csv", index=False)

print(event_log)





           PatientID    Activity               Timestamp   OPO Gender  \
77991     OPO1_P1000    Referral 2033-06-21 22:45:30.263  OPO1      M   
77992     OPO1_P1000  Evaluation 2033-06-21 22:46:30.263  OPO1      M   
43614   OPO1_P100009    Referral 2034-08-11 15:01:25.340  OPO1      F   
43615   OPO1_P100009  Evaluation 2034-08-11 15:02:25.340  OPO1      F   
59914    OPO1_P10004    Referral 2036-04-13 00:02:01.093  OPO1      M   
...              ...         ...                     ...   ...    ...   
282445  OPO6_P999899  Evaluation 2034-07-01 05:08:00.000  OPO6      F   
276298  OPO6_P999923    Referral 2031-09-20 09:06:00.000  OPO6      F   
276299  OPO6_P999923  Evaluation 2031-09-20 09:07:00.000  OPO6      F   
266318   OPO6_P99994    Referral 2030-08-23 05:14:00.000  OPO6      M   
266319   OPO6_P99994  Evaluation 2030-08-23 05:15:00.000  OPO6      M   

                     Race   Age  
77991   White / Caucasian  15.0  
77992   White / Caucasian  15.0  
43614   White / Cauca